

Deep neural networks are on the spotlight on the search for a realistic artificial intelligence. Among many methods, generative adversarial networks (GANs) have shown astonishing results in generative related tas	ks. These problems scale up pretty quickly and classical computers struggle to keep up. 
On the other hand, quantum computation is known to provide enormous speed ups for many problems, especially those related directly with linear algebra (cita). And even though hardware is still in an early stage, there exist already many practical near term developments , such as VQE. Neural networks are really efficient even with a small number of nodes, as long as it is deep enough. Low qubits systems could provide very good results if enough gates are used.

QGANs is the quantum version of GANs. The idea is the same: generate new datasets which follow certain patterns. In order to do this, GANs implement two competing networks: a discriminator and a generator. On the one hand the discriminator has the task to determine which datasets are real and which are false. On the other hand the generator tries to fool the discriminator with the datasets that it generates. Both networks are trained and compete, until reaching an equilibrium with non-zero sum. At this point, the generator is able to create datasets with a quality such that it fools a very well trained discriminator. 
QGANs work exactly the same way. We will be working with quantum states as dataset (which might even represent embedded classical data, as we will see later on), and trainable quantum circuits, which will represent the generators and discriminators. We will train parameters from the circuit in order to discriminate and generate new states efficiently. Moreover, we will introduce labeled dataset in order to generate different types of results. For instance, with labeled data we would be able to generate datasets with different properties on demand.

We will train the system in two steps, where each step is performed in a different quantum device. In the first step, we will train the discriminator by updating its parameters and minimizing the cost function given by 

Cost = Prob(Detect True|Fake) - Prob(Detect true|true)

This quantity is measured as the Sz projection on the Out D register and is runned on cirq.simulator.
On the second step, we will train the generator by minimizing the cost function
Cost = - Prob(Detect True|Fake)
This quantity is also measured by the Sz projection on the Out D register but with a different circuit construction, and is runned on qiskit.aer.

Both the generator and the discriminator are built with similar layered structure. Each layer consists of single qubits Rx(theta1) and Rz(theta2) rotations, and an entangling gate RZZ(theta3) between each qubit and the following.

Our circuit was built for generating eigenstates of Sz. The label will dictate whether the desired state is a |0> state or a |1> state. In order to accomplish this, we would expect that our generating circuit applies a control not operation conditioned on the labeled. This behavior should emerge from the minimization of the angles. The total system has 4 qubits: Out D, Label D, Out R|G and Label R|G. Two of these qubits go into the generator, and three (we reuse Out R|G) to the discriminator. 
We recreated the results from the original paper, where 4 layers were employed for the discriminator and 2 for the generator, accounting for 42 total angle parameters. 



In [1]:
# ! pip install qiskit
# ! pip install cirq
# ! pip install pennylane
# ! pip install pennylane-qiskit
# ! pip install pennylane-cirq

In [2]:
import pennylane as qml
import numpy as np
import tensorflow as tf
import math
from pennylane.templates import AngleEmbedding


In [3]:
# Define generic layer template for both the generator and discriminator
@qml.template
def QGAN_Ansatz(rot_x, rot_z, rot_zz, wires):
  n = len(wires)
  AngleEmbedding(rot_x, wires, rotation='X')
  AngleEmbedding(rot_z, wires, rotation='Z')
  for i in range(math.ceil((n - 1) / 2)):
    qml.MultiRZ(rot_zz[i], wires=[2 * i, 2 * i + 1])
  for j in range(math.ceil((n - 2) / 2)):
    qml.MultiRZ(rot_zz[j + math.ceil((n - 1) / 2)], wires=[2 * j + 1, 2 * (j + 1)])


In [4]:
# Instantiate devices
dev1 = qml.device('cirq.simulator', wires=4)
dev2 = qml.device('qiskit.aer', wires=4)

In [6]:
# Initialize random seed
np.random.seed(0)

In [21]:
# Define different circuit components:
# Label: determines in a potentially random manner wether the intended state is a |0> or a |1>
# Real: generates the real state from the label
# Generator: 2-layered ansatz circuit to be modified to mimic the real circuit
# Discriminator: 4-layered ansatz circuit to be modified to detect fraudulent states generated by the generator
def label(sign=None, **kwargs):
    if sign is not None:
      qml.RX(np.pi * sign, wires=3)
      qml.RX(np.pi * sign, wires=1)
    elif np.random.random() > 0.5:
      qml.RX(np.pi, wires=3)
      qml.RX(np.pi, wires=1)

def real(**kwargs):
    qml.CNOT(wires=[3, 2])

def generator(rot_x, rot_z, rot_zz, wires, layers, **kwargs):
    for i in range(layers):
      QGAN_Ansatz(rot_x[i,:], rot_z[i,:], rot_zz[i,:], wires)

def discriminator(rot_x, rot_z, rot_zz, wires, layers, **kwargs):
    for i in range(layers):
      QGAN_Ansatz(rot_x[i,:], rot_z[i,:], rot_zz[i,:], wires)
    

In [8]:
# Define both global circuits: real and generated
@qml.qnode(dev1, interface="tf")
def real_disc_circuit(rot_disc_x, rot_disc_z, rot_disc_zz):
    label()
    real()
    discriminator(rot_disc_x, rot_disc_z, rot_disc_zz, [0, 1, 2], 4)
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev2, interface="tf")
def gen_disc_circuit(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz):
    label()
    generator(rot_gen_x, rot_gen_z, rot_gen_zz, [2, 3], 2)
    discriminator(rot_disc_x, rot_disc_z, rot_disc_zz, [0, 1, 2], 4)
    return qml.expval(qml.PauliZ(0))

In [9]:
def prob_real_true(rot_disc_x, rot_disc_z, rot_disc_zz):
    true_disc_output = real_disc_circuit(rot_disc_x, rot_disc_z, rot_disc_zz)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz):
    fake_disc_output = gen_disc_circuit(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true

# Cost functions for training both the discriminator and the generator
def disc_cost(rot_disc_x, rot_disc_z, rot_disc_zz):
    cost = prob_fake_true(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz) - prob_real_true(rot_disc_x, rot_disc_z, rot_disc_zz)
    return cost


def gen_cost(rot_gen_x, rot_gen_z, rot_gen_zz):
    return -prob_fake_true(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz)

In [10]:
# Initialize layers initial parameters
init_rot_gen_x = np.random.normal(size=(2, 2))
init_rot_gen_z = np.random.normal(size=(2, 2))
init_rot_gen_zz = np.random.normal(size=(2, 1))

init_rot_disc_x = np.random.normal(size=(4, 3))
init_rot_disc_z = np.random.normal(size=(4, 3))
init_rot_disc_zz = np.random.normal(size=(4, 2))

rot_gen_x = tf.Variable(init_rot_gen_x)
rot_gen_z = tf.Variable(init_rot_gen_z)
rot_gen_zz = tf.Variable(init_rot_gen_zz)

rot_disc_x = tf.Variable(init_rot_disc_x)
rot_disc_z = tf.Variable(init_rot_disc_z)
rot_disc_zz = tf.Variable(init_rot_disc_zz)

In [11]:
opt = tf.keras.optimizers.SGD(0.4)


In [12]:
# Train discriminator
cost = lambda: disc_cost(rot_disc_x, rot_disc_z, rot_disc_zz)

for step in range(50):
    opt.minimize(cost, [rot_disc_x, rot_disc_z, rot_disc_zz])
    if step % 5 == 0:
      cost_val = cost().numpy()
      print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 0.0018607527017593384
Step 5: cost = 0.024176612496376038
Step 10: cost = -0.5029317736625671
Step 15: cost = -0.42606474459171295
Step 20: cost = -0.012887947261333466
Step 25: cost = -0.5378648935584351
Step 30: cost = -0.5014202994061634
Step 35: cost = -0.5239300158573315
Step 40: cost = -0.38021469581872225
Step 45: cost = -0.5725376507034525


In [13]:
print("Prob(real classified as real): ", prob_real_true(rot_disc_x, rot_disc_z, rot_disc_zz).numpy())


Prob(real classified as real):  0.9984251469140872


In [14]:
print("Prob(fake classified as real): ", prob_fake_true(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz).numpy())


Prob(fake classified as real):  0.4443359375


In [15]:
# Train generator
cost = lambda: gen_cost(rot_gen_x, rot_gen_z, rot_gen_zz)

for step in range(35):
    opt.minimize(cost, [rot_gen_x, rot_gen_z, rot_gen_zz])
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.3974609375
Step 5: cost = -0.2958984375
Step 10: cost = -0.53515625
Step 15: cost = -0.654296875
Step 20: cost = -0.076171875
Step 25: cost = -0.9697265625
Step 30: cost = -0.9677734375


In [16]:
print("Prob(fake classified as real): ", prob_fake_true(rot_gen_x, rot_gen_z, rot_gen_zz, rot_disc_x, rot_disc_z, rot_disc_zz).numpy())


Prob(fake classified as real):  0.0263671875


In [17]:
print("Prob(real classified as real): ", prob_real_true(rot_disc_x, rot_disc_z, rot_disc_zz).numpy())


Prob(real classified as real):  0.9984251469140872


In [18]:
print("Discriminator cost: ", disc_cost(rot_disc_x, rot_disc_z, rot_disc_zz).numpy())


Discriminator cost:  -0.04324592463672161


In [19]:
@qml.template
def real_template(l, **kwargs):
  label(l)
  real()

@qml.template
def generator_template(l, **kwargs):
  label(l)
  generator(rot_gen_x.numpy(), rot_gen_z.numpy(), rot_gen_zz.numpy(), [2, 3], 2)


In [20]:
obs = [qml.PauliX(2), qml.PauliY(2), qml.PauliZ(2)]

bloch_vector_real = qml.map(real_template, obs, dev1, interface="tf")
bloch_vector_generator = qml.map(generator_template, obs, dev2, interface="tf")

print("Real Bloch vector with label |0>: {}".format(bloch_vector_real(0)))
print("Generator Bloch vector with label |0>: {}".format(bloch_vector_generator(0)))

print("Real Bloch vector with label |1>: {}".format(bloch_vector_real(1)))
print("Generator Bloch vector with label |1>: {}".format(bloch_vector_generator(1)))

Real Bloch vector with label |0>: [0. 0. 1.]
Generator Bloch vector with label |0>: [0.1328125  0.25       0.95117188]
Real Bloch vector with label |1>: [ 0.  0. -1.]
Generator Bloch vector with label |1>: [0.16796875 0.26953125 0.94726562]
